In [ ]:
# default_exp core

In [ ]:
import nbdev.showdoc as literacy

In [ ]:
#export
from speechsep.imports import *
import speechsep.utils as utils
import speechsep.plot as plot

# Core

This contains most of the basic functions and spectrogram class types. To visualize the spectrograms we will also include a special color map since this makes it easier to notice differences in audio intensities.

The most important things to remember are
- How to create an AudioItem both from a numpy array and from file.
- Creating a SpecImage and how the parameters influence the final result.
- Basic SpecImage Visualizer (more indepth explanation here***)

In [ ]:
#export
@delegates(load)
def load_audio(fn, **kwargs):
    return load(fn)

In [ ]:
fn = Path("/home/holydemon/Music/LJ001-0001.wav")
sig, sr = load_audio(fn)
display(Audio(sig, rate=sr))

## AudioItem

In [ ]:
#export
class AudioItem():
    def __init__(self, filename=None, signal=None, sample_rate=None):
        self.fn = filename
        if signal is None         : self.sig, self._sr = load_audio(self.fn)
        else                      : self.sig, self._sr = signal, sample_rate
        if sample_rate:
            self.sr = sample_rate
        elif self.sr is None:
            raise ValueError('sample_rate must be calculated or given')
    def __repr__(self):
        display(Audio(self.sig, rate=self.sr))
        return f'{self.__str__()}'
    def __str__(self):
        return f'{self.fn}, {len(self.sig)/self.sr}secs at {self.sr} samples per second'
    @delegates(Line2D)
    def show(self, **kwargs):
        plt.plot(self.sig, **kwargs)
    @property
    def sr(self): return self._sr
    @sr.setter
    def sr(self, new_sr):
        if self._sr != new_sr:
            self.sig = utils.Resample(new_sr)(self.sig, self.sr)
        self._sr = new_sr

In [ ]:
audio = AudioItem(fn)
audio.show()
audio

In [ ]:
audio.sr = audio.sr//10
audio.show()
audio

## Spectrograms

In [ ]:
#export
def toSpec(audio:AudioItem):
    pass

In [ ]:
#export
class SpecBase():
    def __init__(self, spec, sr, name=None):
        self.data = spec
        self.sr = sr
        self.name = name
    @classmethod
    def create(cls, fn, sr=None, name=None):
        "Open an `Audio` from path `fn`"
        if isinstance(fn,(Path,str)): return cls.create(AudioItem(fn, sr), name=name)
        elif isinstance(fn,AudioItem): return toSpec(fn.sig, fn.sr, name)
        elif isinstance(fn,np.ndarray): return cls(fn, sr, name)
        raise ValueError('fn must be ndarray, AudioItem or Path')

In [ ]:
    def show(self, ctx=None, **kwargs):
        plot.setup_graph(title=title, x_label=x_label, y_label=y_label, fig_size=fig_size)
        self._plot(abs(spec[:spec.shape[0]//2]))
    @property
    def plt_params(self): return self._plt_params
    @delegate(plt.pcolormesh)
    @plt_params.setter
    def plt_params(self, **kwargs):
        self._plot = partial(plt.pcolormesh, **kwargs)
        self._plt_params = dict(**kwargs)